In [8]:
import pandas as pd
import numpy as np

In [9]:
data = pd.read_csv('/home/kirill/Downloads/Telegram Desktop/code_blocks_final.csv', sep='\t')
data.head()

,code_block,tag
0,\nimport pandas as pd\nimport matplotlib.pyplo...,['invite people for the kaggle party']
1,"\ndf_train = pd.read_csv('../input/train.csv'),",['bring in the six packs']
2,"\ndf_train.columns,",['check the decoration']
3,"\ndf_train['SalePrice'].describe(),",['descriptive statistics summary']
4,"\nsns.distplot(df_train['SalePrice']);,",['histogram']


Уникальных тегов:

In [10]:
data['tag'].nunique()

128808

Размер:

In [11]:
data['tag'].shape

(270661,)

In [12]:
from collections import Counter
from tqdm import tqdm

unique_comms = Counter(data['tag']).most_common()

In [13]:
data.shape

(270661, 2)

In [14]:
all_my_data = pd.DataFrame(columns=['code_block', 'tag'])
for comment in tqdm(unique_comms):
    uniq_cbs = data[data['tag']==comment[0]]['code_block'].unique()
    #print(uniq_cbs)
    # data.drop(data[data['tag'] == comment[0]].index, inplace=True)
    my_data = pd.DataFrame(
        np.array([uniq_cbs, np.repeat(comment[0], len(uniq_cbs))]).T, columns=['code_block', 'tag']
    )
    #print(my_data)
    #if all_my_data.isnull:
    #    all_my_data = my_data.copy()
    #else:
    all_my_data = all_my_data.append(my_data, ignore_index=True)

100%|██████████| 128809/128809 [51:03<00:00, 42.05it/s] 


In [15]:
all_my_data.to_csv('code_blocks_final_clean.csv')

In [16]:
data = all_my_data

In [72]:
data = pd.read_csv('code_blocks_final_clean.csv')
data

,Unnamed: 0,code_block,tag
0,0,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,"[' plotting', ' linear algebra', ' accessing d..."
1,1,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,"[' plotting', ' linear algebra', ' accessing d..."
2,2,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,"[' plotting', ' linear algebra', ' accessing d..."
3,3,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,"[' plotting', ' linear algebra', ' accessing d..."
4,4,from sklearn.preprocessing import StandardScal...,"[' plotting', ' linear algebra', ' accessing d..."
...,...,...,...
221798,221798,"\nclf_list = [('Perceptron', Perceptron(eta0=0...",[' list of classifiers to compare use some ad...
221799,221799,"def test_set_performances(clf_tuple_list, X_tr...",[' fit each model to training data and form pr...
221800,221800,"\nmodel_names, test_acc, test_f1 = test_set_pe...",[' obtain accuracy and f1 metrics and print fo...
221801,221801,"def plot_confusion_matrix(true_y, pred_y, titl...",91 plot a confusion matrix for our trained model


In [76]:
data.shape

(221803, 3)

In [78]:
data = data.drop(columns='Unnamed: 0', axis=1)

### Удаление стоп-слов, стемминг, оставляем по одному комментарию

In [79]:
# from nltk import download
# download('stopwords')

# import nltk
# nltk.download('wordnet')

#import nltk
#nltk.download('punkt')

# import nltk
# nltk.download('averaged_perceptron_tagger')

In [80]:
from nltk.stem.snowball import SnowballStemmer 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [81]:
st_wr = set(stopwords.words('english'))

In [82]:
ex_str = 'kirill eats grass on the floors, long word endings, churches'

lemmatizer thx to 

https://medium.com/@gaurav5430/using-nltk-for-lemmatizing-sentences-c1bfff963258 

(left a clap):

In [83]:
import nltk
from nltk.corpus import wordnet

wnl = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(lemmatizer, sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

lemmatize_sentence(wnl, ex_str)

'kirill eat grass on the floor , long word ending , church'

In [84]:
stemmer = SnowballStemmer('english')
stemmer.stem(ex_str)

'kirill eats grass on the floors, long word endings, church'

**Важно:** если у вас в табличке есть колонка 'Unnamed 0:' или вообще какая то лишняя, то дропайте ее до этого цикла (почему-то это сильно влияет на производительность)

Тут из нескольких комментариев выбирается самый короткий (обосновано тем, что тогда он с большей вероятностью встречается часто ака больше похож на метку). Отсекаются пробелы и всякие пустые метки.

In [85]:
import re
def clear_tag(tag):
    ready_tag = tag.replace('_', ' ')
    ready_tag = re.sub('[^A-Za-z0-9 ]+', '', ready_tag)
    ready_tag = ready_tag.strip()                    # removes spaces in the start and in the end
    ready_tag = ready_tag.replace(' ', '_')
    return ready_tag # replace spaces with "_" and removes other punctuation

In [86]:
from tqdm import tqdm
all_to_del = set(['', "['']", ' ', '  ', "[' ']", '        ', '     ', '    ', '   ', "['', ' ']", '       ', '      ', \
             '             ', '         ', '          ', "']",\
                 "['   ", "['", "[' ", "['  ", "['    ", "['    "])
rows_to_del = []

### Режем по длинне метки:

Здесь проверка, чтобы не было копипаста, потому что комментарий длиннее 95% всех меток (ака ~200 символов) совпавший так много раз явно коппаст:

В среднем слов:

In [95]:
data['tag'].str.split('_').str.len().sum() / data.shape[0]

3.5781131821772525

95-ые квантили:

In [96]:
np.quantile(data['tag'].str.len(), 0.95)

58.0

In [97]:
np.quantile(data['tag'].str.split('_').str.len(), 0.95)

9.0

In [87]:
for i in tqdm(range(data.shape[0])):
    my_string = data.loc[i, 'tag']
    
    if (my_string is not np.nan) and my_string is not None:
        
        min_len = len(data.loc[i, 'tag']) + 10
        min_el = None
        s = set(list(map(str, my_string.split("', '"))))
        
        for el in s:
            
            # getting rid of punctuation, spaces, adding "_"
            el = clear_tag(el)
            
            if el not in all_to_del and el is not None and len(el) < 100:
                
                # getting rid of stop words
                el = [w.lower() for w in el.split('_') if w.lower() not in st_wr]
                el = '_'.join(el)
                
                # stemming 
                el = stemmer.stem(el)
                
                # lemmatizing
                el = lemmatize_sentence(wnl, el)
                
                #if min_el is None:
                #    min_el = el
                #else:
                #    min_el = min_el + '_' + el
                
                if len(el) < min_len:
                    min_len = len(el)
                    min_el = el

        if min_el is not None:
            data.loc[i, 'tag'] = min_el
        else:
            rows_to_del.append(i)
    else:
        rows_to_del.append(i)

100%|██████████| 221803/221803 [03:04<00:00, 1200.36it/s]


Было:

In [88]:
data.shape

(221803, 2)

In [89]:
data = data.drop(rows_to_del)

Осталось:

In [90]:
data.shape

(178782, 2)

In [91]:
data = data.reset_index()

In [92]:
data.drop(columns=['index'], axis=1, inplace=True)
data.head()

,code_block,tag
0,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,plot
1,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,plot
2,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,plot
3,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,plot
4,from sklearn.preprocessing import StandardScal...,plot


________

### Предобработка текста

In [100]:
data = data[data['tag'] != '']
data = data[data['tag'] != '_']
data = data[data['tag'] != '__']
data = data[data['tag'] != '___']
data = data[data['tag'] != '____']
data = data[data['tag'] != '_____']

Осталось уже:

In [101]:
data.shape

(178491, 2)

_____________

In [102]:
print(data.shape)
print(data['tag'].nunique())

(178491, 2)
70994


Дальше часть скипается вплоть до разделения на трейн-тест-вал!

### Оставим какие-то самые популярные метки

Из самых популярных 5к меток выкину варианты длиннее n

In [103]:
data['tag'].nunique()

70994

In [104]:
from collections import Counter

mc = Counter(data['tag']).most_common(5000)

In [105]:
strange_to_del = []

Это работает не очень 

Если уникальных блоков кода с этим комментарием меньше 75 (естественно стоит эту константу пробовать менять), то удалим эту метку тк это скорее всего копипаст:

Так делаем среди самых популярных 5к тегов чтобы не тратить время на совсем не популярные метки

In [106]:
for el in tqdm(mc):
    if data[data['tag']==el[0]]['code_block'].nunique() < 75:
        strange_to_del.append(el[0])

100%|██████████| 5000/5000 [00:42<00:00, 118.07it/s]


In [107]:
for el in tqdm(strange_to_del):
    data = data[data['tag'] != el]

100%|██████████| 4905/4905 [01:31<00:00, 53.86it/s]


Здесь естественно теряется много данных:

In [108]:
print(data.shape)
print(data['tag'].nunique())

(118646, 2)
66089


Заметим, что выкинули 4940 меток из самых популярных 5000, оставим 30 самых популярных:

Ниже

In [110]:
Counter(data['tag']).most_common(50)

[('specify_none_want_read_whole_fil', 2403),
 ('linear_algebra', 1864),
 ('model', 1224),
 ('check_answ', 702),
 ('predict', 673),
 ('train', 605),
 ('load_data', 516),
 ('plot', 513),
 ('visual', 487),
 ('preprocess', 462),
 ('code', 430),
 ('submiss', 406),
 ('feature_engin', 399),
 ('random_forest', 395),
 ('test', 391),
 ('import', 345),
 ('eda', 342),
 ('data_preprocess', 320),
 ('data_clean', 303),
 ('train_model', 289),
 ('xgboost', 287),
 ('data_visu', 285),
 ('logistic_regress', 284),
 ('model_build', 274),
 ('importing_librari', 254),
 ('data', 253),
 ('read_data', 248),
 ('import_librari', 222),
 ('uncom', 217),
 ('exploratory_data_analysi', 217),
 ('fit_model', 211),
 ('loading_data', 210),
 ('decision_tre', 199),
 ('1', 194),
 ('normal', 181),
 ('data_explor', 179),
 ('create_x', 176),
 ('reading_data', 175),
 ('linear_regress', 173),
 ('librari', 172),
 ('input', 171),
 ('fit_rfmodelonfulldata_data_training_data', 170),
 ('evalu', 162),
 ('training_model', 161),
 ('titl',

In [111]:
# all_to_del - задан вначале ноутбука

In [112]:
for to_del in all_to_del:
    data = data[data['tag'] != to_del]

In [135]:
print(data.shape)
print(data['tag'].nunique())

(19048, 2)
50


In [137]:
num_tags_to_drop = data['tag'].nunique() - 50
num_tags_to_drop

25

Выкидываю самые непопулярные (nunique - 50) меток:

In [138]:
least_c_tags = set(Counter(data['tag']).most_common()[-num_tags_to_drop:])

In [139]:
for el in tqdm(least_c_tags):
    data = data[data['tag'] != el[0]]

100%|██████████| 25/25 [00:00<00:00, 405.51it/s]


Смотрим что осталось:

In [117]:
print(data.shape)
print(data['tag'].nunique())

(19048, 2)
50


In [118]:
ready_data = \
data.reset_index().drop(columns=['index'], axis=1)
#ready_data = ready_data.drop(columns=['level_0'], axis=1)

In [119]:
mc = Counter(ready_data['tag']).most_common()

In [120]:
with open("most_common_50_1.txt", 'w') as output:
    for row in mc:
        output.write(str(row) + '\n')

In [121]:
mc

[('specify_none_want_read_whole_fil', 2403),
 ('linear_algebra', 1864),
 ('model', 1224),
 ('check_answ', 702),
 ('predict', 673),
 ('train', 605),
 ('load_data', 516),
 ('plot', 513),
 ('visual', 487),
 ('preprocess', 462),
 ('code', 430),
 ('submiss', 406),
 ('feature_engin', 399),
 ('random_forest', 395),
 ('test', 391),
 ('import', 345),
 ('eda', 342),
 ('data_preprocess', 320),
 ('data_clean', 303),
 ('train_model', 289),
 ('xgboost', 287),
 ('data_visu', 285),
 ('logistic_regress', 284),
 ('model_build', 274),
 ('importing_librari', 254),
 ('data', 253),
 ('read_data', 248),
 ('import_librari', 222),
 ('uncom', 217),
 ('exploratory_data_analysi', 217),
 ('fit_model', 211),
 ('loading_data', 210),
 ('decision_tre', 199),
 ('1', 194),
 ('normal', 181),
 ('data_explor', 179),
 ('create_x', 176),
 ('reading_data', 175),
 ('linear_regress', 173),
 ('librari', 172),
 ('input', 171),
 ('fit_rfmodelonfulldata_data_training_data', 170),
 ('evalu', 162),
 ('training_model', 161),
 ('titl',

На данном этапе все работает очень плохо, поэтому давайте попробуем руками с метками поработать...

In [140]:
ready_data['tag'].unique()

array(['plot', 'linear_algebra', 'model', 'check_answ', 'load_data',
       'train', 'exploratory_data_analysi', 'predict', 'import_librari',
       'visual'], dtype=object)

In [124]:
jut_inds = ready_data[ready_data['tag'] == 'specify_none_want_read_whole_fil'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)
jut_inds = ready_data[ready_data['tag'] == 'create_x'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)
jut_inds = ready_data[ready_data['tag'] == 'uncom'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)



ready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'
ready_data.loc[ready_data['tag'] == 'data_load', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'importing_data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'reading_data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'eda', 'tag'] = 'exploratory_data_analysi'
ready_data.loc[ready_data['tag'] == 'data_explor', 'tag'] = 'exploratory_data_analysi'



In [127]:
jut_inds = ready_data[ready_data['tag'] == 'code'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)
jut_inds = ready_data[ready_data['tag'] == '1'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)

ready_data.loc[ready_data['tag'] == 'importing_librari', 'tag'] = 'import_librari'
ready_data.loc[ready_data['tag'] == 'import', 'tag'] = 'import_librari'
ready_data.loc[ready_data['tag'] == 'librari', 'tag'] = 'import_librari'
ready_data.loc[ready_data['tag'] == 'training_model', 'tag'] = 'import_librari'


In [131]:
mc = Counter(ready_data['tag']).most_common()

In [132]:
with open("most_common_50_2.txt", 'w') as output:
    for row in mc:
        output.write(str(row) + '\n')

In [141]:
ready_data

,code_block,tag
0,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,plot
1,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,plot
2,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,plot
3,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,plot
4,from sklearn.preprocessing import StandardScal...,plot
...,...,...
19036,df3=df[df.columns]\nimport warnings \nwarning...,load_data
19038,\nimport numpy as np \nimport pandas as pd \ni...,linear_algebra
19041,\nimport numpy as np \nimport pandas as pd \ni...,linear_algebra
19042,\nimport numpy as np \nimport pandas as pd \ni...,linear_algebra


Без всего:

In [52]:
# data -> load_data
# exploratory_data_analysis, data_exploration -> eda
# data_preprocessing -> preprocessing
# training -> train
# check_your_answer, check_your_answers -> check_answer
# model -> model_building
# just_uncomment_them -> trash


##################### maybe

# prediction - test ?

In [ ]:
ready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'
ready_data.loc[ready_data['tag'] == 'modelling', 'tag'] = '

In [78]:
model'
ready_data.loc[ready_data['tag'] == 'model_building', 'tag'] = 'model_building'
ready_data.loc[ready_data['tag'] == 'data_visualization', 'tag'] = 'visualization'
ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'exploratory_data_analysis', 'tag'] = 'eda'
ready_data.loc[ready_data['tag'] == 'training', 'tag'] = 'train'

jut_inds = ready_data[ready_data['tag'] == 'just_uncomment_them'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)
jut_inds = ready_data[ready_data['tag'] == 'check_your_answer'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)
jut_inds = ready_data[ready_data['tag'] == 'check_your_answers'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)
jut_inds = ready_data[ready_data['tag'] == 'your_code_here'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)

Вариант для лемматизации:

In [79]:
'''ready_data.loc[ready_data['tag'] == 'check_your_answers', 'tag'] = 'check_your_answer'
ready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'
ready_data.loc[ready_data['tag'] == 'training', 'tag'] = 'train'
ready_data.loc[ready_data['tag'] == 'exploratory_data_analysis', 'tag'] = 'eda'
ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'model_building', 'tag'] = 'model'


jut_inds = ready_data[ready_data['tag'] == 'just_uncomment_them'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)
jut_inds = ready_data[ready_data['tag'] == 'your_code_here'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)'''

"ready_data.loc[ready_data['tag'] == 'check_your_answers', 'tag'] = 'check_your_answer'\nready_data.loc[ready_data['tag'] == 'data_preprocessing', 'tag'] = 'preprocessing'\nready_data.loc[ready_data['tag'] == 'training', 'tag'] = 'train'\nready_data.loc[ready_data['tag'] == 'exploratory_data_analysis', 'tag'] = 'eda'\nready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'\nready_data.loc[ready_data['tag'] == 'model_building', 'tag'] = 'model'\n\n\njut_inds = ready_data[ready_data['tag'] == 'just_uncomment_them'].index\nready_data.drop(index=jut_inds, axis=0, inplace=True)\njut_inds = ready_data[ready_data['tag'] == 'your_code_here'].index\nready_data.drop(index=jut_inds, axis=0, inplace=True)"

Вариант для стемминга:

In [80]:
# train, train_model, model_build, model, fit_model
# train_model, fit_model -> train
# 'data_visu' -> visual
# model_building -> model_build

In [81]:
'''ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'
ready_data.loc[ready_data['tag'] == 'train_model', 'tag'] = 'train'
ready_data.loc[ready_data['tag'] == 'fit_model', 'tag'] = 'train'
ready_data.loc[ready_data['tag'] == 'data_preprocess', 'tag'] = 'preprocess'
ready_data.loc[ready_data['tag'] == 'check_answers', 'tag'] = 'check_answer'
ready_data.loc[ready_data['tag'] == 'model', 'tag'] = 'model_building'
ready_data.loc[ready_data['tag'] == 'data_visu', 'tag'] = 'visual'
ready_data.loc[ready_data['tag'] == 'model_building', 'tag'] = 'model_build'

jut_inds = ready_data[ready_data['tag'] == 'uncomment_them'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)'''

"ready_data.loc[ready_data['tag'] == 'data', 'tag'] = 'load_data'\nready_data.loc[ready_data['tag'] == 'train_model', 'tag'] = 'train'\nready_data.loc[ready_data['tag'] == 'fit_model', 'tag'] = 'train'\nready_data.loc[ready_data['tag'] == 'data_preprocess', 'tag'] = 'preprocess'\nready_data.loc[ready_data['tag'] == 'check_answers', 'tag'] = 'check_answer'\nready_data.loc[ready_data['tag'] == 'model', 'tag'] = 'model_building'\nready_data.loc[ready_data['tag'] == 'data_visu', 'tag'] = 'visual'\nready_data.loc[ready_data['tag'] == 'model_building', 'tag'] = 'model_build'\n\njut_inds = ready_data[ready_data['tag'] == 'uncomment_them'].index\nready_data.drop(index=jut_inds, axis=0, inplace=True)"

In [142]:
print(ready_data.shape)
print(ready_data['tag'].nunique())
Counter(ready_data['tag']).most_common()

(9197, 2)
10


[('linear_algebra', 1864),
 ('load_data', 1237),
 ('model', 1224),
 ('import_librari', 1154),
 ('exploratory_data_analysi', 738),
 ('check_answ', 702),
 ('predict', 673),
 ('train', 605),
 ('plot', 513),
 ('visual', 487)]

Слишком много линейной алгебры и др, давайте из них выберем по 800 объектов (не костыль, а эксперимент)

Удаляем unbalance:

In [143]:
todel_mc1 = np.random.choice(
    np.array(ready_data[ready_data['tag'] == 'linear_algebra'].index),
    1864 - 1200,
    replace = False
)
'''todel_mc2 = np.random.choice(
    np.array(ready_data[ready_data['tag'] == 'plot'].index),
    2955 - 1000,
    replace = False
)'''

ready_data.drop(index=todel_mc1, inplace=True, axis=0)
# ready_data.drop(index=todel_mc2, inplace=True, axis=0)

In [144]:
print(ready_data.shape)
print(ready_data['tag'].nunique())
Counter(ready_data['tag']).most_common()

(8533, 2)
10


[('load_data', 1237),
 ('model', 1224),
 ('linear_algebra', 1200),
 ('import_librari', 1154),
 ('exploratory_data_analysi', 738),
 ('check_answ', 702),
 ('predict', 673),
 ('train', 605),
 ('plot', 513),
 ('visual', 487)]

In [170]:
ready_data[ready_data['tag'] == 'load_data']['code_block'][2010]

"DATA_PATH = '../input/CORD-19-research-challenge/'\nCLEAN_DATA_PATH = '../input/cord-19-eda-parse-json-and-generate-clean-csv/'\npmc_df = pd.read_csv(CLEAN_DATA_PATH + 'clean_pmc.csv')\nbiorxiv_df = pd.read_csv(CLEAN_DATA_PATH + 'biorxiv_clean.csv')\ncomm_use_df = pd.read_csv(CLEAN_DATA_PATH + 'clean_comm_use.csv')\nnoncomm_use_df = pd.read_csv(CLEAN_DATA_PATH + 'clean_noncomm_use.csv')\npapers_df = pd.concat([pmc_df,\n                       biorxiv_df,\n                       comm_use_df,\n                       noncomm_use_df], axis=0).reset_index(drop=True),"

In [186]:
jut_inds = ready_data[ready_data['tag'] == 'specify_none_want_read_whole_fil'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)

jut_inds = ready_data[ready_data['tag'] == 'maturitysize1_size_maturity_1__small_2__medium_3__large_4__extra_large_0__specifi'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)

jut_inds = ready_data[ready_data['tag'] == 'check_answ_fill_line_get_names_columns_missing_valu_fill_lines_drop_columns_training_validation_data_code'].index
ready_data.drop(index=jut_inds, axis=0, inplace=True)

In [208]:
mc = Counter(ready_data['tag']).most_common()
for el in tqdm(mc):
    if 'check' in el[0]:
        to_del_ind = ready_data[ready_data['tag'] == el[0]].index
        ready_data.drop(index=to_del_ind, axis=0, inplace=True)

100%|██████████| 74863/74863 [01:25<00:00, 880.30it/s] 


In [209]:
print(ready_data.shape)
print(ready_data['tag'].nunique())
Counter(ready_data['tag']).most_common()

(126576, 3)
71836


[('model', 629),
 ('predict', 359),
 ('train', 287),
 ('random_forest', 272),
 ('feature_engin', 255),
 ('submiss', 240),
 ('load_data', 239),
 ('preprocess', 202),
 ('data_preprocess', 200),
 ('eda', 170),
 ('visual', 163),
 ('fit_rfmodelonfulldata_data_training_data_improve_accuracy_create_new_random_forest_model_train_training_data',
  156),
 ('model_build', 156),
 ('import', 151),
 ('test', 149),
 ('xgboost', 146),
 ('data_visu', 142),
 ('importing_librari', 141),
 ('exploratory_data_analysi', 137),
 ('data_processing_csv_file_io_eg_pdreadcsv_linear_algebra', 131),
 ('logistic_regress', 125),
 ('data_explor', 122),
 ('import_librari', 120),
 ('loading_data', 113),
 ('train_model', 107),
 ('decision_tre', 106),
 ('data_load', 105),
 ('result', 105),
 ('importing_data', 103),
 ('read_data', 103),
 ('linear_regress', 99),
 ('data_clean', 99),
 ('data', 90),
 ('train_test_split', 89),
 ('training_model', 86),
 ('reading_data', 85),
 ('librari', 80),
 ('import_data', 79),
 ('feature_sel

____________

Соотношения train, test, val = 8.5 / 1.5 / 1.5

Поделим нацело:

In [149]:
(ready_data.shape[0] - 9) / 11

774.0

In [150]:
ready_data = ready_data.drop(ready_data.tail(9).index)

In [151]:
ready_data.shape[0] / 11

774.0

То есть размер трейна:

In [152]:
774.0 * 8

6192.0

In [153]:
ready_data['tag']

0             plot
1             plot
2             plot
3             plot
4             plot
           ...    
18991    load_data
18992    load_data
18993    load_data
18994        train
18995        train
Name: tag, Length: 8514, dtype: object

Стратифайдом делю на трейн, тест, сплит

In [155]:
# https://stackoverflow.com/questions/40829137/stratified-train-validation-test-split-in-scikit-learn

from sklearn.model_selection import StratifiedShuffleSplit
SEED = 42001

split = StratifiedShuffleSplit(n_splits=1, test_size=3/11, random_state=SEED)
for train_index, test_valid_index in split.split(\
                                                 ready_data.drop('tag', axis=1), \
                                                 ready_data['tag']\
                                                ):
    train_set = ready_data.iloc[train_index]
    test_valid_set = ready_data.iloc[test_valid_index]

split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=SEED)
for test_index, valid_index in split2.split(test_valid_set, test_valid_set['tag']):
    test_set = test_valid_set.iloc[test_index]
    valid_set = test_valid_set.iloc[valid_index]

In [156]:
train_set = train_set.reset_index()
test_set = test_set.reset_index()
valid_set = valid_set.reset_index()

______

Далее все '\n' заменяются на '\n\t', а также вначале каждого сниппета приписывается

'def tag():\n\t'

Каждый сниппет записывается в **отдельный файл** с названием номер_итерации.py в каталог трей, тест или валидейшн.

In [157]:
def add_tabs(exmpl):
    new_exmpl = str()
    for i in range(len(exmpl)):
        if (exmpl[i]=='\n'):
            new_exmpl += '\n\t'
        else:
            new_exmpl += exmpl[i]
    return new_exmpl

train:

In [158]:
for i in range(train_set.shape[0]):
        exmpl = train_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(train_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_train/func'+str(i)+'.py', 'w'))

test:

In [159]:
for i in range(test_set.shape[0]):
        exmpl = test_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(test_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_test/func'+str(i)+'.py', 'w'))

valid:

In [160]:
for i in range(valid_set.shape[0]):
        exmpl = valid_set.loc[i]['code_block'][:-1]
        exmpl = add_tabs(exmpl)
        print('def ' + str(valid_set.loc[i]['tag'])+'():\n\t' + exmpl
              ,  file=open('/home/kirill/code2vec/cd2vec/python/my_val/func'+str(i)+'.py', 'w'))

In [161]:
len(valid_set)

1161

_______

Предобработка для просто 30 строк кода посложнее, пока ее нет.